In [2]:
if Sys.iswindows()
    username = "Alex"
    pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
    using Pkg
    Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
    using StaticArrays, DifferentialEquations, DynamicalSystems, BenchmarkTools, CairoMakie
    include("C:\\Users\\Alex\\Desktop\\dynamical-systems\\system.jl");
    include("C:\\Users\\Alex\\Desktop\\dynamical-systems\\visual.jl");
else
    username = "nova"
    pathtorepo = "/home/nova/work/repo_ds/dynamical-systems"
    using Pkg
    Pkg.activate(pathtorepo * "/env/integrate/")
    using StaticArrays, DifferentialEquations, DynamicalSystems, JLD2, CairoMakie
    include("/home/nova/work/repo_ds/dynamical-systems/system.jl")
    include("/home/nova/work/repo_ds/dynamical-systems/visual.jl")
end

  Activating project at `~/work/repo_ds/dynamical-systems/env/integrate`


plot_3d_2c_fp (generic function with 1 method)

In [3]:
function find_zero(Λs; tol = 1e-3)
    vec_bool = Vector{Bool}(undef, 6);
    for (index, lp) in enumerate(Λs)
        checknull=  isapprox(lp, 0.0; atol = tol);
        vec_bool[index] = checknull;
    end
    vec_of_true = findall(x->x==true, vec_bool);
    if length(vec_of_true) >=1
        return 0;
    else
        return -1;
    end;
end

function lyapunovspectrum_adapt(tands, time_lse; tol_ = 1e-3)
    coeff_adapt_time = 1;
    flag = true;

    Λs = lyapunovspectrum(tands, time_lse);
    while flag == true
        checkzero = find_zero(Λs; tol = tol_);
        if checkzero == -1
            coeff_adapt_time*=2;
            Λs = lyapunovspectrum(tands, time_lse * coeff_adapt_time*10);
        else
            flag = false;
        end
    end
    return Λs, coeff_adapt_time;
end

lyapunovspectrum_adapt (generic function with 1 method)

In [4]:
time = 3000.0; tt = 0.0; tstep = 0.001; tol = 1e-15;
tlse = 1000;
integ_set = (alg = Vern9(), adaptive = true, abstol = tol, reltol = tol);

In [5]:
param = TM6_glial_ECM_get_params();
dict, indexparams = TM6_glial_ECM_help(param);
println(indexparams);


    τ - 1, τD - 2, τF - 3, τy - 4, α - 5, αE - 6, αecm - 7, αp - 8,
    J - 9, U0 - 10, I0 - 11, ΔU0 - 12, β - 13, βecm  -14, βp - 15,
    γp - 16, ecm0 - 17, ecm1 - 18, kecm - 19, θecm - 20, p0 - 21, p1 - 22, θp - 23, kp - 24, ythr - 25, xthr - 26


In [6]:
u0 = [9.229913037372262, 0.4700671097035878, 0.8142744230362869, 0.00014525973293301472, 0.0074559249055014525, 1.42904680196444e-16];
param[6] = 4.6235863453815265

4.6235863453815265

In [7]:
ds= CoupledODEs(TM6_glial_ECM, u0, param, diffeq = integ_set);
tr, trange = trajectory(ds, time, Δt = tstep)

(6-dimensional StateSpaceSet{Float64} with 3000001 points, 0.0:0.001:3000.0)

In [8]:
tands = TangentDynamicalSystem(ds; J = TM6_glial_ECM_jac);

In [9]:
tands

6-dimensional TangentDynamicalSystem
 deterministic:     true
 discrete time:     false
 in-place:          false
 dynamic rule:      TM6_glial_ECM
 jacobian:          TM6_glial_ECM_jac
 deviation vectors: 6
 parameters:        [0.013, 0.15, 1.0, 1.8, 1.5, 4.6235863453815265, 0.001, 0.01, 3.07, 0.3  …  0.0, 1.0, 0.15, 25.6, 0.0, 1.0, 26.0, 0.05, 0.5, 0.9]
 time:              0.0
 state:             [9.093154846317159, 0.47441434243155256, 0.8122299370061847, 0.00015847507922404021, 0.004548511197734886, 7.443368064550399e-11]


In [10]:
Λs, _ =  lyapunovspectrum_adapt(tands, tlse);
Λs

6-element Vector{Float64}:
 -0.0008632545732685082
 -0.013218160645093692
 -0.2802714009350248
 -0.4121473477179941
 -0.7071293022523626
 -3.5540092494219286

In [24]:
#plot_timesereis(trange, tr[:, 1], 1, 3000000, width = 2000, height = 300, inter=false, lw = 1.0, color = :deeppink)